In [1]:
import pandas as pd
import numpy as np

import requests
import openpyxl as xl
import time
import json

import folium
import re

import geopandas as gpd
from matplotlib import pyplot as plt # 그래프 그리는 용도
import seaborn as sns
import geopandas as gpd
from plotnine import *
from shapely.geometry import Point, Polygon

import warnings
warnings.filterwarnings(action='ignore') # 경고메세지 숨기기

In [28]:
df = pd.read_excel("fin_dataset_dong.xlsx")
# df2 = pd.read_excel("fin_dataset_goo.xlsx")
df = df.rename(columns={"goo_nm":"goo_nm"})

In [29]:
df.head(1)

,goo_nm,dong_ID,dong_nm,dong_area,sum_pop,sum_hh,one_hh,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station,sum_order,sum_waste,y,x
0,송파구,11710631,가락1동,1.46,27876,9534,1045,0,0,1,0,10,2,2605376,6434890,73694,4271.9,37.496537,127.105726


In [30]:
g_df = df[["goo_nm","sum_waste","sum_order"]].drop_duplicates(["goo_nm"])
g_df = g_df.sort_values(["goo_nm"], ascending=True)
g_df = g_df.reset_index()
g_df = g_df.drop(columns={"index"})
g_df.head()

,goo_nm,sum_waste,sum_order
0,강남구,5556.9,68999
1,강동구,2766.3,50316
2,강북구,838.3,45685
3,강서구,4928.8,86172
4,관악구,1631.4,129233


In [31]:
sub = df[["goo_nm","dong_area","sum_pop","sum_hh","one_hh","cc_value","count_zero","count_market","count_univ","count_bus","count_station","sum_bus","sum_station"]]
sub.head()

,goo_nm,dong_area,sum_pop,sum_hh,one_hh,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station
0,송파구,1.46,27876,9534,1045,0,0,1,0,10,2,2605376,6434890
1,송파구,1.34,33204,11482,1892,10,0,2,0,22,2,2181971,3449427
2,송파구,0.96,27368,10576,2642,30,1,1,0,25,0,3081377,0
3,구로구,0.40,14263,4901,2244,10,0,0,0,10,0,2235038,0
4,금천구,2.52,25973,14124,8016,30,0,6,0,105,2,10824582,14509425


In [32]:
condition = ["goo_nm","dong_area","sum_pop","sum_hh","one_hh","cc_value","count_zero","count_market","count_univ","count_bus","count_station","sum_bus","sum_station"]
goo_df2 = sub.groupby(by = ['goo_nm'])[condition].transform(sum)
goo_df2 = goo_df2.drop_duplicates(["goo_nm"])
goo_df2 = goo_df2.sort_values(["goo_nm"], ascending=True)
goo_df2 = goo_df2.reset_index()

g_df2 = goo_df2.drop(columns={"index"})
g_df2=g_df2.rename(columns={"goo_nm":"지역구"})
g_df2.head(1)

,지역구,dong_area,sum_pop,sum_hh,one_hh,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station
0,강남구강남구강남구강남구강남구강남구강남구강남구강남구강남구강남구강남구강남구강남구강남구강...,39.51,544055,208833,58663,457,0,46,1,552,22,102636377,107653238


In [33]:
goo_df = pd.concat([g_df,g_df2], axis = 1)
goo_df = goo_df.rename(columns = {"dong_area":"goo_area"})
goo_df = goo_df.drop(columns={"지역구"})
goo_df

,goo_nm,sum_waste,sum_order,goo_area,sum_pop,sum_hh,one_hh,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station
0,강남구,5556.9,68999,39.51,544055,208833,58663,457,0,46,1,552,22,102636377,107653238
1,강동구,2766.3,50316,24.58,463998,177148,39486,414,3,21,0,396,10,34783737,34193728
2,강북구,838.3,45685,23.6,311569,129805,32979,320,0,17,0,412,11,57437812,25855961
3,강서구,4928.8,86172,41.44,585901,244097,69879,548,1,22,2,619,16,55236175,43691763
4,관악구,1631.4,129233,29.57,509803,248959,115556,460,2,24,1,490,6,89607537,61049638
5,광진구,1584.9,62301,17.06,360109,152090,54100,343,1,23,3,292,9,32782959,48074042
6,구로구,1278.5,50971,20.12,432488,164083,40152,187,0,24,3,480,10,60262348,53472371
7,금천구,3794.8,42334,13.02,247835,103432,34714,126,0,20,0,354,3,44554050,18515469
8,노원구,1108.3,56438,35.56,527032,202695,38700,441,1,19,5,535,12,49940954,45054621
9,도봉구,1290.8,34884,20.46,327361,127362,24178,370,0,12,1,366,6,38499802,22486646


In [34]:
goo_df.head(5)

,goo_nm,sum_waste,sum_order,goo_area,sum_pop,sum_hh,one_hh,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station
0,강남구,5556.9,68999,39.51,544055,208833,58663,457,0,46,1,552,22,102636377,107653238
1,강동구,2766.3,50316,24.58,463998,177148,39486,414,3,21,0,396,10,34783737,34193728
2,강북구,838.3,45685,23.6,311569,129805,32979,320,0,17,0,412,11,57437812,25855961
3,강서구,4928.8,86172,41.44,585901,244097,69879,548,1,22,2,619,16,55236175,43691763
4,관악구,1631.4,129233,29.57,509803,248959,115556,460,2,24,1,490,6,89607537,61049638


In [35]:
goo_df["one_ratio"]=""

In [37]:
for i in range(25):
    goo_df["one_ratio"][i]=goo_df["one_hh"][i]/goo_df["sum_pop"][i]

In [38]:
goo_df

,goo_nm,sum_waste,sum_order,goo_area,sum_pop,sum_hh,one_hh,cc_value,count_zero,count_market,count_univ,count_bus,count_station,sum_bus,sum_station,one_ratio
0,강남구,5556.9,68999,39.51,544055,208833,58663,457,0,46,1,552,22,102636377,107653238,0.107825
1,강동구,2766.3,50316,24.58,463998,177148,39486,414,3,21,0,396,10,34783737,34193728,0.0851
2,강북구,838.3,45685,23.6,311569,129805,32979,320,0,17,0,412,11,57437812,25855961,0.105848
3,강서구,4928.8,86172,41.44,585901,244097,69879,548,1,22,2,619,16,55236175,43691763,0.119268
4,관악구,1631.4,129233,29.57,509803,248959,115556,460,2,24,1,490,6,89607537,61049638,0.226668
5,광진구,1584.9,62301,17.06,360109,152090,54100,343,1,23,3,292,9,32782959,48074042,0.150232
6,구로구,1278.5,50971,20.12,432488,164083,40152,187,0,24,3,480,10,60262348,53472371,0.09284
7,금천구,3794.8,42334,13.02,247835,103432,34714,126,0,20,0,354,3,44554050,18515469,0.140069
8,노원구,1108.3,56438,35.56,527032,202695,38700,441,1,19,5,535,12,49940954,45054621,0.07343
9,도봉구,1290.8,34884,20.46,327361,127362,24178,370,0,12,1,366,6,38499802,22486646,0.073857


In [42]:
goo_df.shape

(25, 16)

In [43]:
goo_df.to_excel("fin_dataset_goo.xlsx")